## Backtesting using historical prices

In [1]:
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
from numpy import dot
from numpy import divide
from numpy.linalg import multi_dot as mdot
from numpy.linalg import inv
import matplotlib.pyplot as plt
from matplotlib import rc
import quadprog
import yfinance
import os
from weights import Portfolio
np.set_printoptions(precision = 3, suppress = True, linewidth = 400)

### Downloading historical data

In [2]:
tickers = ['SPY', 'XLF', 'EEM', 'EWZ', 'SLV', 'FXI', 'GDX', 'HYG', 'TLT', 'XLU']
#etfs = yfinance.download(tickers, auto_adjust = True, start = "2011-9-1", end='2019-9-1')['Close']
etfs = yfinance.download(tickers, start = "2011-9-1")['Close']

[*********************100%***********************]  10 of 10 completed


In [3]:
etfs = etfs/etfs.iloc[0]
etfs = etfs.pct_change().iloc[1:]*100

In [4]:
etfs.to_csv("../data/etfs.csv", sep=";")

## Steps in GARCH backtesting

---- Inital setup----

$t_{-1}$ is last period/yesterday - all information about this period is known when in period $t$


$t$ is our current period/today - this is where weights are decided

$t_{+1}$ is next period/tomorrow - we attempt to forecast the volatility of this period


0. Fit model
1. Receive variables and parse them: sigma ($\sigma$), dcca (a), dccb (b), residuals ($\epsilon$), alpha ($\alpha$), beta ($\beta$), omega ($\omega$)
---- Enter loop ---- 

0. Get variables from current period for all assets: $\epsilon^2_{t}$, $\sigma^2_{t}$
1. Calculate for all assets $\epsilon_t = r_t - \mu$
2. Calculate for all assets $ \sigma_{i,t+1}^2=\omega_i+\alpha_i\epsilon_{i,t}^2+\beta_i\sigma_{i,t}^2$
3. Calculate $Var_{t+1} = diag(\sigma_{t+1}^2)$
4. Calculate $\eta_t = inv(Var_t)*\epsilon_t$
5. If first loop: Calculate Qbar = $\frac{1}{T}\sum_{t=1}^T\eta_t\eta_t'$
5. Calculate $Q_{t+1} = Qbar*(1-dcca-dccb) + dcca*\eta_{t}*\eta_{t}' + b*Q_{t}$
6. Calculate $Q_{t+1}^{*-1} = diag(Q_{t+1}^{-1})$ 
7. Calculate $\Gamma_{t+1}=Q_{t+1}^{*-1}Q_{t+1}Q_{t+1}^{*-1}$
8. Calculate $\Omega_{t+1}=\text{Var}_{t+1}\Gamma_{t+1}\text{Var}_{t+1}$

## Obtain params from R

### Fitting initial model

In [5]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()
os.chdir('/Users/nielseriksen/thesis/')
# Defining the R script and loading the instance in Python
r = ro.r
r['source']('backtesting/fitting_mgarch.R')

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


### Initial setup

In [46]:
# 0. Fit model
length_sample_period = len(etfs)-100
out_of_sample = etfs.iloc[length_sample_period:,]
# Loading the function we have defined in R.
fit_mgarch_r = ro.globalenv['fit_mgarch']
#Fitting the mgarch model and receiving the result
coef, residuals, sigmas = fit_mgarch_r(length_sample_period)

In [31]:
#Receive variables from model
asset_names = etfs.columns.values

# How elegant
mu, o, al, be = np.hsplit(coef[:-2].reshape((len(asset_names), 4)), 4)
mu, o, al, be = map(np.ravel, (mu, o, al, be))  # Flattening to 1d array
dcca = coef[-2]
dccb = coef[-1]
epsilons = residuals

### Enter loop

In [35]:
Omega_ts = []
for t, r_t in enumerate(out_of_sample.values):
    
    # 0. Get values from last period
    e_t_1, e_t_1_squared, s_t_1_squared = values_from_last_period(residuals, sigmas)

    # 1. Calculate current period epsilon
    e_t, epsilons = calculate_epsilon(epsilons, r_t, mu)

    # 2. Calculate for all assets sigma_^2
    s_t_squared = calculate_sigma(o, al, e_t_squared, be, s_t_squared)

    # 3. Calculate Var_t, Var_t_inv
    Var_t_plus_1, Var_t_plus_1_inv = calculate_Var_t(s_t_squared)
    
    # 4. Calculate eta_t
    eta_t = dot(Var_t_plus_1_inv, e_t)
    
    # 5. If first period calculate the constant Qbar
    if t == 0:
        # 5. Calculate Qbar
        Qbar = calculate_Qbar(epsilons, sigmas)
        Q_t = Qbar
    
    # 6. Calculate Q_t_plus_1
    Q_t_plus_1 = calculate_Q_t_plus_1(Qbar, dcca, dccb, eta_t, Q_t)
    
    # 7. Calculate Q_t_plus_1^{*-1}
    Q_t_plus_1_s_inv = calculate_Q_t_plus_1_s_inv(Q_t_plus_1)
    
    # 8. Calulate Gamma_t
    Gamma_t_plus_1 = calculate_Gamma_t_plus_1(Q_t_plus_1_s_inv, Q_t_plus_1)
    
    # 9. 
    Omega_t_plus_1 = calculate_Omega_t_plus_1(Var_t_plus_1, Gamma_t_plus_1)
    
    # Storing
    Omega_ts.append(Omega_t_plus_1)
    sigmas = np.append(sigmas, s_t_squared, axis = 0)
    
    # Iterate on period
    #Q_t_1 = Q_t 
    #eta_t_1 = eta_t    

(10,)
[9.868 9.876 9.869 9.884 9.873 9.869 9.868 9.891 9.873 9.883]
[[9.868 0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    9.876 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    9.869 0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    9.884 0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    9.873 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    9.869 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    9.868 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    9.891 0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    9.873 0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    9.883]]


LinAlgError: 1-dimensional array given. Array must be two-dimensional

In [39]:
def values_from_last_period(epsilons, sigmas):
    e_t_1 = epsilons[-1]
    e_t_1_squared = epsilons[-1]**2
    s_t_1_squared = sigmas[-1]
    return e_t_1, e_t_1_squared, s_t_1_squared

In [40]:
def calculate_epsilon(r_t, mu):
    e_t = np.array([r_t - mu]).T
    epsilons = np.append(epsilons, e_t.T, axis=0)
    return e_t, epsilons

In [42]:
def calculate_sigma(o, al, e_t_1_squared, be, s_t_1_squared):
    next_sigma = np.array([o + al*e_t_1_squared + be*s_t_1_squared])
    return next_sigma

In [50]:
def calculate_Qbar(epsilons, sigmas):
    eta = []
    for epsilon_t, sigma_t in zip(epsilons, sigmas):
        eta.append(dot(inv(calc_Var_t(sigma_t)), epsilon_t))

    eta = np.array(eta)
    Qbar = 1/len(epsilons) * sum([dot(eta, eta.T) for eta in eta])
    return Qbar

In [51]:
def calculate_Var_t(s_t_squared):
    Var_t = np.diag(np.ravel(s_t_squared))
    Var_t_inv = inv(Var_t)
    return Var_t, Var_t_inv

In [53]:
def calulate_Q_t_plus_1(Qbar, dcca, dccb, eta_t, Q_t):
    Q_t_plus_1 = np.array(Qbar*(1-dcca-dccb) + dcca*eta_t*eta_t.T + dccb*Q_t)
    return Q_t_plus_1

In [55]:
def calculate_Q_t_plus_1_s_inv(Q_t_plus_1):
    Q_t_plus_1_s_inv = inv(np.diag(Q_t_plus_1))
    return Q_t_plus_1_s_inv

In [57]:
def calculate_Gamma_t_plus_1(Q_t_plus_1_s_inv, Q_t_plus_1):
    Gamma_t_plus_1 = mdot([Q_t_plus_1_s_inv, Q_t_plus_1, Q_t_plus_1_s_inv])
    return Gamma_t_plus_1

In [58]:
def calculate_Omega_t_plus_1(Var_t_plus_1, Gamma_t_plus_1):
    Omega_t_plus_1 = mdot([Var_t_plus_1, Gamma_t_plus_1, Var_t_plus_1])
    return Omega_t_plus_1




### Misc 

In [21]:
rcov_forecast_r = ro.globalenv['rcov_forecast']
rcov = rcov_forecast_r()[0][0]

In [22]:
tri = np.zeros((10, 10))
tri[np.triu_indices(n = 10, k=0, m=10)] = rcov

In [45]:
Omega = tri + tri.T - np.diag(np.diag(tri))

In [46]:
Omega

array([[ 1.263,  1.834,  1.579,  0.73 ,  0.118,  0.684,  0.329, -0.208,  0.429,  0.028],
       [ 1.834,  6.118,  1.778,  1.244,  0.202,  1.234,  0.596, -0.437,  0.905,  0.055],
       [ 1.579,  1.778,  2.576,  0.547,  0.1  ,  0.576,  0.294, -0.241,  0.392, -0.088],
       [ 0.73 ,  1.244,  0.547,  3.702,  0.149,  2.394,  0.322,  0.131,  0.31 ,  0.15 ],
       [ 0.118,  0.202,  0.1  ,  0.149,  0.052,  0.104,  0.083, -0.012,  0.095,  0.062],
       [ 0.684,  1.234,  0.576,  2.394,  0.104,  2.61 ,  0.231,  0.032,  0.204, -0.012],
       [ 0.329,  0.596,  0.294,  0.322,  0.083,  0.231,  0.268, -0.095,  0.321,  0.185],
       [-0.208, -0.437, -0.241,  0.131, -0.012,  0.032, -0.095,  0.523, -0.294,  0.093],
       [ 0.429,  0.905,  0.392,  0.31 ,  0.095,  0.204,  0.321, -0.294,  0.735,  0.166],
       [ 0.028,  0.055, -0.088,  0.15 ,  0.062, -0.012,  0.185,  0.093,  0.166,  0.938]])